<a href="https://colab.research.google.com/github/KhuyenLE-maths/Implementation-of-GoogLeNet-on-Keras/blob/main/Implementation_of_GoogLeNet_on_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I. Giới thiệu
GoogLeNet là một mạng nơ-ron tích chập sâu được đề xuất bởi [Szegedy và các cộng sự.](https://www.cv-foundation.org/openaccess/content_cvpr_2015/html/Szegedy_Going_Deeper_With_2015_CVPR_paper.html) . Mạng này đã giành chiến thắng trong Thử thách nhận dạng hình ảnh quy mô lớn ImageNet 2014 (ILSVRC-2014) với hiệu suất phân loại đạt 92,3%. Đặc biệt, mô hình này được thiết kế theo kiến trúc đặc biệt cho phép tăng độ sâu và chiều rộng của mạng nhưng vẫn giữ nguyên tài nguyên tính toán.

Mô hình VGG có tổng cộng 22 lớp và bao gồm 9 khối Inception. Mỗi khối Inception bao gồm bốn đường dẫn song song mà tại đó các lớp tích chập với các kích thước kernel khác nhau:
- Đường dẫn đầu tiên sử dụng một lớp tích chập với kích thước là 1 × 1.
- Trong đường dẫn thứ hai và thứ ba, một lớp tích chập có kích thước 1 × 1 được sử dụng trước khi đi qua lớp 3 × 3 và 5 × 5. Lớp tích chập 1 × 1 giúp giảm số kênh bộ lọc, do đó giảm độ phức tạp của mô hình.
- Đường dẫn thứ tư sử dụng một lớp max-pooling để giảm độ phân giải của đầu vào và tiếp theo là một lớp tích chập 1 × 1 để giảm chiều.

Bốn đường dẫn này sử dụng phần đệm thích hợp để đầu vào và đầu ra có cùng kích thước. Việc concat bốn đường dẫn này cho phép quét đầu vào ở các độ phân giải khác nhau. Đặc biệt, độ phức tạp của mô hình được giảm thiểu nhờ áp dụng lớp tích chập 1 × 1 trong mỗi đường dẫn.


![title](Image/InceptionBlock.png)

Có thể tóm tắt như sau:
- Kích thước hình ảnh đầu vào là 224 × 224.
- Có chín khối Inception trong mạng này.
- Có bốn lớp max-pooling bên ngoài các khối Inception, trong đó có hai lớp nằm giữa các khối 3–4 và khối 7–8. Các lớp max-pooling này giúp giảm kích thước của dữ liệu đầu vào, do đó giảm độ phức tạp của mô hình cũng như chi phí tính toán.
- Mạng này kế thừa ý tưởng sử dụng lớp pooling trung bình từ NiN, giúp cải thiện hiệu suất mô hình và giảm hiện tượng overfitting.
- Một lớp dropout (với 40%) được sử dụng trước lớp linear. Đây cũng là một phương pháp regularization hiệu quả để giảm hiện tượng overfitting.
- Lớp đầu ra sử dụng hàm kích hoạt softmax để đưa ra 1000 đầu ra tương ứng với số lượng danh mục trong tập dữ liệu ImageNet.


Ngoài ra, một số mạng bổ sung được thêm vào bên cạnh, giúp khuyến khích sự phân biệt ở các giai đoạn thấp hơn trong bộ phân loại, tăng tín hiệu gradient nhận được backpropagation và cung cấp regularization bổ sung. Cấu trúc của các mạng này bao gồm:
- Một lớp pooling trung bình với kích thước pooling là 5 × 5 và stride là 3.
- Một lớp tích chập 1 × 1 với 128 bộ lọc để giảm chiều và kích hoạt RELU.
- Một lớp fully connected với 1024 units và kích hoạt RELU.
- Một dropout với tỷ lệ 70% đầu ra.
- Một lớp đầu ra sử dụng hàm kích hoạt softmax để phân loại đối tượng thành một trong 1000 classes.



![title](Image/TableStructure.png)

# II. Triển khai GoogLeNet trên Keras

Đầu tiên chúng ta cần import một số thư viện:

In [3]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers import concatenate

Tạo một Inception block 

In [4]:
def Inception_block(input_layer, f1, f2_conv1, f2_conv3, f3_conv1, f3_conv5, f4): 
  # Input: 
  # - f1: số lượng bộ lọc của lớp tích chập 1x1 trong đường dẫn đầu tiên
  # - f2_conv1, f2_conv3 là số lượng bộ lọc tương ứng với các lớp tích chập 1x1 và 3x3 trong đường dẫn thứ hai
  # - f3_conv1, f3_conv5 là số lượng bộ lọc tương ứng với lớp tích chập 1x1 và 5x5 trong đường dẫn thứ ba
  # - f4: số lượng bộ lọc của lớp tích chập 1x1 trong đường dẫn thứ tư

  # 1st path:
  path1 = Conv2D(filters=f1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)

  # 2nd path
  path2 = Conv2D(filters = f2_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path2 = Conv2D(filters = f2_conv3, kernel_size = (3,3), padding = 'same', activation = 'relu')(path2)

  # 3rd path
  path3 = Conv2D(filters = f3_conv1, kernel_size = (1,1), padding = 'same', activation = 'relu')(input_layer)
  path3 = Conv2D(filters = f3_conv5, kernel_size = (5,5), padding = 'same', activation = 'relu')(path3)

  # 4th path
  path4 = MaxPooling2D((3,3), strides= (1,1), padding = 'same')(input_layer)
  path4 = Conv2D(filters = f4, kernel_size = (1,1), padding = 'same', activation = 'relu')(path4)

  output_layer = concatenate([path1, path2, path3, path4], axis = -1)

  return output_layer


Hàm để triển khai mô hình GoogLeNet:

In [5]:
def GoogLeNet():
  # input layer 
  input_layer = Input(shape = (224, 224, 3))

  # convolutional layer: filters = 64, kernel_size = (7,7), strides = 2
  X = Conv2D(filters = 64, kernel_size = (7,7), strides = 2, padding = 'valid', activation = 'relu')(input_layer)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # convolutional layer: filters = 64, strides = 1
  X = Conv2D(filters = 64, kernel_size = (1,1), strides = 1, padding = 'same', activation = 'relu')(X)

  # convolutional layer: filters = 192, kernel_size = (3,3)
  X = Conv2D(filters = 192, kernel_size = (3,3), padding = 'same', activation = 'relu')(X)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 1st Inception block
  X = Inception_block(X, f1 = 64, f2_conv1 = 96, f2_conv3 = 128, f3_conv1 = 16, f3_conv5 = 32, f4 = 32)

  # 2nd Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 192, f3_conv1 = 32, f3_conv5 = 96, f4 = 64)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size= (3,3), strides = 2)(X)

  # 3rd Inception block
  X = Inception_block(X, f1 = 192, f2_conv1 = 96, f2_conv3 = 208, f3_conv1 = 16, f3_conv5 = 48, f4 = 64)

  # Extra network 1:
  X1 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X1 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X1)
  X1 = Flatten()(X1)
  X1 = Dense(1024, activation = 'relu')(X1)
  X1 = Dropout(0.7)(X1)
  X1 = Dense(5, activation = 'softmax')(X1)

  
  # 4th Inception block
  X = Inception_block(X, f1 = 160, f2_conv1 = 112, f2_conv3 = 224, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 5th Inception block
  X = Inception_block(X, f1 = 128, f2_conv1 = 128, f2_conv3 = 256, f3_conv1 = 24, f3_conv5 = 64, f4 = 64)

  # 6th Inception block
  X = Inception_block(X, f1 = 112, f2_conv1 = 144, f2_conv3 = 288, f3_conv1 = 32, f3_conv5 = 64, f4 = 64)

  # Extra network 2:
  X2 = AveragePooling2D(pool_size = (5,5), strides = 3)(X)
  X2 = Conv2D(filters = 128, kernel_size = (1,1), padding = 'same', activation = 'relu')(X2)
  X2 = Flatten()(X2)
  X2 = Dense(1024, activation = 'relu')(X2)
  X2 = Dropout(0.7)(X2)
  X2 = Dense(1000, activation = 'softmax')(X2)
  
  
  # 7th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, 
                      f3_conv5 = 128, f4 = 128)

  # max-pooling layer: pool_size = (3,3), strides = 2
  X = MaxPooling2D(pool_size = (3,3), strides = 2)(X)

  # 8th Inception block
  X = Inception_block(X, f1 = 256, f2_conv1 = 160, f2_conv3 = 320, f3_conv1 = 32, f3_conv5 = 128, f4 = 128)

  # 9th Inception block
  X = Inception_block(X, f1 = 384, f2_conv1 = 192, f2_conv3 = 384, f3_conv1 = 48, f3_conv5 = 128, f4 = 128)

  # Global Average pooling layer 
  X = GlobalAveragePooling2D(name = 'GAPL')(X)

  # Dropoutlayer 
  X = Dropout(0.4)(X)

  # output layer 
  X = Dense(1000, activation = 'softmax')(X)
  
  # model
  model = Model(input_layer, [X, X1, X2], name = 'GoogLeNet')

  return model

In [6]:
model = GoogLeNet()

In [7]:
model.summary()

Model: "GoogLeNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 109, 109, 64  9472        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 54, 54, 64)   0           ['conv2d[0][0]']                 
                                                                                          

Mặc dù mô hình phức tạp để triển khai, số lượng tham số của toàn bộ mô hình không lớn. Các lớp Dense luôn chiếm phần lớn các tham số. Bên cạnh đó, sự xuất hiện của các lớp pooling trung bình toàn cục giúp giảm đáng kể số lượng tham số, do đó làm giảm độ phức tạp tính toán của mô hình.

# IV. Kết luận
Chúng em đã triển khai mô hình GoogLeNet trên nền tảng Keras. Nó bao gồm các khối Inception. Mỗi khối có một kiến ​​trúc đặc biệt, trong đó nó trích xuất đồng thời các đặc điểm đầu vào thông qua bốn đường dẫn song song. Bên cạnh đó, các lớp tích chập 1×1 được áp dụng tối đa trong các đường dẫn này để giảm chiều kênh. Hơn nữa, việc áp dụng lớp max-pooling giữa một số khối Inception đóng vai trò làm giảm độ phân giải, do đó làm giảm độ phức tạp tính toán. Tóm lại, số lượng tham số của mô hình này nhỏ hơn 6 lần so với mô hình Alexnet và nhỏ hơn nhiều so với mô hình VGG. Đặc biệt, nó vượt trội hơn các mô hình này. Kiến trúc thú vị này của GoogLeNet cũng là nguồn cảm hứng cho sự xuất hiện của các mô hình sau này

**Tài liệu tham khảo:**

[1] Szegedy, Christian, et al. “Going deeper with convolutions.” Proceedings of the IEEE conference on computer vision and pattern recognition. 2015.